In [ ]:
# Constructing returns data from prices
# Author: Prof. Paul Goldsmith-Pinkham
# Course: MGT 544 - Investment Management

import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Set plotting style
sns.set_palette("husl")

# Return Calculations and Risk Analysis

This notebook explores fundamental concepts in measuring investment returns and risk:

1. **Return Calculations**
   - Simple vs. Log returns
   - Properties and differences
   
2. **Risk Measures**
   - Traditional measures (volatility, skewness, kurtosis)
   - Downside risk measures (VaR, Expected Shortfall)
   
3. **Time-Varying Risk**
   - Rolling statistics
   - Market regimes

Let's start by calculating and comparing returns data. We'll use the S&P 500 index as an example.

In [ ]:
# Download data and calculate returns
def get_returns(ticker='SPY', start='2010-01-01'):
    print(f"Downloading data for {ticker}...")
    prices = yf.download(ticker, start=start)['Adj Close']
    
    returns = pd.DataFrame()
    returns['simple_ret'] = prices.pct_change() * 100
    returns['log_ret'] = np.log(prices/prices.shift(1)) * 100
    returns['price'] = prices
    
    return returns

returns = get_returns()

# Plot 1: Price and Returns Over Time
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Price plot
ax1.plot(returns.index, returns['price'])
ax1.set_title('SPY Price Over Time')
ax1.set_ylabel('Price ($)')
ax1.grid(True)

# Returns plot
ax2.plot(returns.index, returns['simple_ret'], alpha=0.5, label='Daily Returns')
ax2.set_title('Daily Returns Over Time')
ax2.set_ylabel('Returns (%)')
ax2.grid(True)
plt.tight_layout()
plt.show()


# Plot 2: Compare Simple vs Log Returns
plt.figure(figsize=(12, 6))
sns.histplot(data=returns[['simple_ret', 'log_ret']].melt(), 
             x='value', hue='variable', stat='density', common_norm=False, alpha=0.5)
plt.title('Distribution of Returns: Simple vs Log')
plt.xlabel('Returns (%)')
plt.ylabel('Density')

# Add normal distribution for comparison
x = np.linspace(returns['simple_ret'].min(), returns['simple_ret'].max(), 100)
plt.plot(x, stats.norm.pdf(x, returns['simple_ret'].mean(), returns['simple_ret'].std()),
         'r--', label='Normal Distribution')
plt.legend()
plt.show()


# Statistical Analysis
def print_stats(data, name):
    stats = pd.DataFrame(columns=['Value'])
    stats.loc['Mean (%)'] = f"{data.mean():.2f}"
    stats.loc['Std Dev (%)'] = f"{data.std():.2f}"
    stats.loc['Skewness'] = f"{data.skew():.2f}"
    stats.loc['Kurtosis'] = f"{data.kurtosis():.2f}"
    stats.loc['Annualized Return (%)'] = f"{data.mean() * 252:.2f}"
    stats.loc['Annualized Vol (%)'] = f"{data.std() * np.sqrt(252):.2f}"
    print(f"\n{name} Return Statistics:")
    print(stats)


print_stats(returns['simple_ret'], 'Simple')
print_stats(returns['log_ret'], 'Log')

# Arithmetic vs. Geometric Returns

Key concepts we'll explore:

1. **Arithmetic Returns**
   - Simple average of returns
   - Used for forecasting future returns
   - Easier to work with mathematically

2. **Geometric Returns**
   - Compound rate of return
   - Reflects actual investor experience
   - More appropriate for historical performance

3. **Annualization**
   - Allows comparison across different time periods
   - Accounts for compounding
   - Industry standard for reporting


In [ ]:
# Download data
spy = yf.download('SPY', start='2000-01-01')['Adj Close']
returns = pd.DataFrame()
returns['simple_ret'] = spy.pct_change()

# Calculate different holding period returns
# Daily, Weekly, Monthly, Quarterly, Annual
def calculate_returns(prices, periods=[1, 5, 21, 63, 252]):
    results = pd.DataFrame(index=['Arithmetic', 'Geometric', 'Difference'],
                           columns=[f'{p}-day' for p in periods])

    for p in periods:
        # Calculate p-period returns
        rets = prices.pct_change(p).dropna()

        # Arithmetic mean (convert to annual)
        arith_mean = rets.mean() * (252/p)

        # Geometric mean (convert to annual)
        geom_mean = ((1 + rets).prod()) ** (252/(p*len(rets))) - 1

        results[f'{p}-day']['Arithmetic'] = arith_mean
        results[f'{p}-day']['Geometric'] = geom_mean
        results[f'{p}-day']['Difference'] = arith_mean - geom_mean

    return results * 100  # Convert to percentages


results = calculate_returns(spy)
print("\nAnnualized Returns Comparison:")
print(results.round(2))


# Demonstration of why geometric is more accurate for realized returns
print("\nSimple Example with Two Periods:")
example_rets = pd.Series([0.10, -0.10])  # +10% followed by -10%
arith_mean = example_rets.mean()
geom_mean = (1 + example_rets).prod() ** (1/len(example_rets)) - 1

print(f"Period 1 Return: +10%")
print(f"Period 2 Return: -10%")
print(f"Arithmetic Average: {arith_mean:.1%}")
print(f"Geometric Average: {geom_mean:.1%}")
print(f"Actual Final Value of $100: ${
      100 * (1 + example_rets[0]) * (1 + example_rets[1]):.2f}")
print(f"Using Arithmetic: ${100 * (1 + arith_mean) ** 2:.2f}")
print(f"Using Geometric: ${100 * (1 + geom_mean) ** 2:.2f}")

# Visualization of compounding effects
plt.figure(figsize=(12, 6))
periods = 10
rates = pd.DataFrame({
    'Arithmetic': [100 * (1 + arith_mean) ** i for i in range(periods)],
    'Geometric': [100 * (1 + geom_mean) ** i for i in range(periods)],
    'Actual': [100 * ((1 + example_rets[0]) * (1 + example_rets[1])) ** (i/2) for i in range(periods)]
})

rates.plot(title='Growth of $100 Using Different Return Calculations')
plt.ylabel('Portfolio Value ($)')
plt.xlabel('Number of Periods')
plt.grid(True)
plt.show()

   
## Key Observations:

1. **Arithmetic vs. Geometric**
   - Arithmetic mean ≥ Geometric mean (Jensen's inequality)
   - Difference increases with volatility
   - Geometric better reflects actual investor experience

2. **Time Horizon Effects**
   - Longer horizons show larger differences
   - Volatility impacts compound returns
   - Arithmetic mean overestimates long-term growth

3. **Business Applications**
   - Use geometric for past performance
   - Use arithmetic for future projections
   - Consider horizon and volatility in analysis

In [ ]:

# Additional demonstration: Impact of volatility on arithmetic-geometric difference
def simulate_return_differences(mean_ret, vol, periods=252, simulations=1000):
    differences = []
    for _ in range(simulations):
        # Simulate daily returns
        rets = np.random.normal(mean_ret/periods, vol /
                                np.sqrt(periods), periods)

        arith_mean = np.mean(rets) * periods
        geom_mean = np.prod(1 + rets) ** (1/periods) - 1
        geom_mean = (1 + geom_mean) ** periods - 1

        differences.append(arith_mean - geom_mean)

    return np.mean(differences)


# Compare difference at various volatility levels
vols = np.arange(0.1, 0.5, 0.1)
diffs = [simulate_return_differences(0.08, vol) for vol in vols]

plt.figure(figsize=(10, 6))
plt.plot(vols, diffs)
plt.title('Impact of Volatility on Arithmetic-Geometric Return Difference')
plt.xlabel('Annual Volatility')
plt.ylabel('Return Difference (Arithmetic - Geometric)')
plt.grid(True)
plt.show()

Finally, let's look at some risk measures that we can calculate from the returns data that are often used in risk management.

In [ ]:

# Risk Measures Analysis
def calculate_risk_measures(returns, confidence_level=0.05):
    VaR = np.percentile(returns.dropna(), confidence_level*100)
    ES = returns[returns <= VaR].mean()

    # Calculate historical probabilities of various returns
    prob_negative = (returns < 0).mean() * 100
    prob_minus_2 = (returns < -2).mean() * 100

    risk_stats = pd.DataFrame(columns=['Value'])
    risk_stats.loc[f'{confidence_level*100}% VaR (%)'] = f"{VaR:.2f}"
    risk_stats.loc[f'{confidence_level *
                      100}% Expected Shortfall (%)'] = f"{ES:.2f}"
    risk_stats.loc['Probability of Negative Return (%)'] = f"{
        prob_negative:.1f}"
    risk_stats.loc['Probability of Return < -2% (%)'] = f"{prob_minus_2:.1f}"

    return risk_stats, VaR, ES


risk_stats, VaR, ES = calculate_risk_measures(returns['simple_ret'])
print("\nRisk Measures:")
print(risk_stats)

# Plot 3: Returns Distribution with Risk Measures
plt.figure(figsize=(12, 6))
sns.histplot(returns['simple_ret'].dropna(),
             stat='density', alpha=0.5, label='Returns')
plt.axvline(x=VaR, color='red', linestyle='--', label='VaR')
plt.axvline(x=ES, color='purple', linestyle='--', label='Expected Shortfall')
plt.title('Return Distribution with Risk Measures')
plt.xlabel('Returns (%)')
plt.ylabel('Density')
plt.legend()
plt.show()

## Key Observations:

1. **Return Properties**:
   - Simple and log returns are very similar for daily data
   - Returns show significant deviation from normality (higher kurtosis)
   - Annualized volatility is consistent with historical patterns

2. **Risk Characteristics**:
   - VaR and Expected Shortfall capture tail risk
   - Historical probability of negative returns shows significant downside risk
   - Returns distribution shows fat tails compared to normal distribution

3. **Business Implications**:
   - Risk management needs to account for fat tails
   - Simple summary statistics may underestimate true risk
   - Time-varying nature of risk requires dynamic management

## Discussion Questions:
1. How would these patterns change across different asset classes?
2. What are the implications for risk management?
3. How should these measures inform portfolio decisions?